# PGT Routing Optimization - Optimization Model

Project student team: Peter Pan; Vincent Pan; Sanjit Sokhi, Jerry Wang, Jiadi Zhang

Advisor: Amr Farahat

Creation date: 2023-03-27

This jupyter notebook performs the network flow optimization based upon the material provided by the material generation notebook

In [343]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime, timedelta
import numpy as np
from geopy.geocoders import Nominatim 
from geopy import distance 
from multidict import MultiDict
import math
import gurobipy as gp
from gurobipy import GRB
import pickle

In [344]:
optimization_time_limit = 60

In [345]:
with open('C:/Users/jiadiz/Desktop/PGT Trucking/initial model/Github/model_materials.pkl', 'rb') as f:
    model_materials = pickle.load(f)

In [346]:
terminals, drivers, jobs, nodes, jobs_revenues, all_arcs, flow_arcs, order_data_set = model_materials 

In [347]:
len(flow_arcs)

35207

In [348]:
def sum_for_all_driver(d,flow_arcs):
    out = []
    for ch in flow_arcs:
        if ch[0] == d:
            out.append(ch)
    return out

In [349]:
def all_driver_value(x,d,flow_arcs):
    out = []
    for ch in flow_arcs:
        
        if ch[0] == d:
            
            out.append(flow_arcs[ch])
    return out

In [350]:
# def find_the_next(path_example,k):
#     driver = k[0]
#     dest = k[2]
#     for ch in path_example:
#         if ch[0] == driver and dest == ch[1]:
#             return ch 
#     return 

In [351]:
def find_the_next(path_example,k):
    driver = k[0]
    dest = k[2]
    for ch in path_example:
        if ch[0] == driver and dest == ch[1]:
            return ch 
    return 

def generated_path(path_example):
    all_path = []
    path = ''
    for ch in path_example:
        driver = ch[0]
        if ch[1][0]!='T':
            continue 
    
        ori = ch[1]
        dest = ch[2]
        path = driver +':'+ch[1] +'->' + ch[2] 
        nex = find_the_next(path_example,ch)
        path_example.remove(ch)
        while nex:
            path = path +'->' + nex[2]
            temp = nex
            path_example.remove(nex)
            nex  = find_the_next(path_example,temp) 
            
        all_path.append(path)
        path = ''
        
    return all_path 

In [352]:
mod = gp.Model('model0')

In [353]:
y = mod.addVars(jobs, vtype=GRB.BINARY, name= "y")
x = mod.addVars(flow_arcs, vtype=GRB.BINARY, name= "x")

In [354]:
obj_fn = mod.setObjective(gp.quicksum(jobs_revenues[j]* y[j] for j in jobs), GRB.MAXIMIZE)

In [355]:
# obj_fn = mod.setObjective(gp.quicksum(y[j] for j in jobs), GRB.MAXIMIZE)

In [356]:
u = mod.addVars(nodes, vtype=GRB.INTEGER, name="u")

In [357]:
terminal_constraint = mod.addConstrs((x.sum(d, '*', t) <=1  for d in drivers for t in terminals))


In [358]:
flow_conservation = mod.addConstrs((x.sum(d, '*', j) == x.sum(d, j, '*') for d in drivers for j in nodes if j not in terminals),name="flow_conservation",
)

In [359]:
single_driver = mod.addConstrs(x.sum('*', o, d) <= 1 for (o, d) in all_arcs)

In [360]:
served_jobs = mod.addConstrs(x.sum('*', '*', j)  == y[j] for j in jobs)

In [361]:
mip_gap = 0.05  # 5% optimality gap

# mod.setParam(GRB.Param.MIPGap, mip_gap)
mod.setParam(GRB.Param.TimeLimit, optimization_time_limit)

Set parameter TimeLimit to value 60


In [362]:
for d in drivers:
    driver =  sum_for_all_driver(d,flow_arcs)
    
    mod.addConstr(gp.quicksum(flow_arcs[ch]* x[ch] for ch in driver) <= 80)

In [363]:
mod.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13792 rows, 35537 columns and 170585 nonzeros
Model fingerprint: 0x3d8fa374
Variable types: 0 continuous, 35537 integer (35357 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+02]
  Objective range  [2e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 10959 rows and 14112 columns
Presolve time: 1.98s
Presolved: 2833 rows, 21425 columns, 82033 nonzeros
Variable types: 0 continuous, 21425 integer (21425 binary)
Found heuristic solution: objective 82986.230233

Root relaxation: objective 2.406742e+05, 9438 iterations, 0.69 seconds (0.62 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

In [364]:
complete_path = []
for i in flow_arcs:
    if (x[i].x >=0.99):
        complete_path.append(i)

In [370]:
complete_job = complete_path.copy()
final_result = generated_path(complete_job)
while complete_job != []:
    final_result+= generated_path(complete_job)
final_result

['Driver181:T12->6289->5071->T12',
 'Driver39:T10->45->7577->8455->1589->T10',
 'Driver109:T21->4929->5406->8170->T21',
 'Driver458:T3->8965->9005->8584->T3',
 'Driver41:T17->4658->6439->T17',
 'Driver210:T27->9701->7312->T27',
 'Driver378:T4->1081->1987->T4',
 'Driver453:T14->3757->6329->7226->T14',
 'Driver390:T15->4850->8792->T15',
 'Driver477:T19->7616->579->T19',
 'Driver352:T24->9119->6624->T24',
 'Driver165:T28->5391->3290->7997->T28',
 'Driver12:T18->3208->3925->4479->T18',
 'Driver82:T9->111->3398->T9',
 'Driver361:T16->3896->2381->T16',
 'Driver91:T22->9420->8808->T22',
 'Driver228:T20->1540->5800->T20',
 'Driver294:T13->3877->4173->T13',
 'Driver311:T2->5593->3641->T2',
 'Driver457:T11->7849->8780->T11',
 'Driver344:T25->2037->3305->T25',
 'Driver130:T26->405->9067->4172->7859->T26',
 'Driver223:T1->7734->9556->T1',
 'Driver331:T29->3213->3436->T29',
 'Driver504:T6->9245->4229->T6',
 'Driver550:T8->4204->2363->2013->T8',
 'Driver247:T5->9567->7479->T5',
 'Driver286:T0->5004-

In [367]:
selected_jobs_count = 0
for i in jobs:
    #if (x[i].x >= 0.99):
        #print(x[i].x)
        if y[i].x > 0.5:
            selected_jobs_count += 1 

In [368]:
print('selected_jobs_count :' +str(selected_jobs_count), 'job_count_in_job_pool :' + str(len(order_data_set)), 'job_selection_rate :' + str(selected_jobs_count/len(order_data_set)))

selected_jobs_count :70 job_count_in_job_pool :150 job_selection_rate :0.4666666666666667


In [369]:
print('optimized_revenue :' +str(mod.ObjVal), 'revenue_in_pool:' + str(order_data_set['LineHaulRevenue'].sum()), 'optimized_revenue_to_pool_revenue_rate :' + str(mod.ObjVal/order_data_set['LineHaulRevenue'].sum()))

optimized_revenue :237094.6502051566 revenue_in_pool:446213.4422679697 optimized_revenue_to_pool_revenue_rate :0.5313480674183083
